**Updated Code from 09/09/2024 to 10/12/24**

All Back End Code.
Takes 1 hour orginal Audio Files, Reduces the noise and creates a new 'reduced audio file', creates a new 1-4 second long audio file for each detected noises based on a certain threshold,

Testing ways to put all this data into a excel sheet to visualize information better.

Started working on a function where user can then verify each detected noise by viewing it's Waveform, Spectrogram, and Audio to confirm if it's a SAW, SAW CALL, or Neither.

# Installing & Importing necessary libraries


In [ ]:
!pip install auditok
#library avaible on github. git it duration and what sound energy it's looking for and marks past that point

!pip install noisereduce
#it reduces the noise in the audio file. noises causes issues during analysis

!pip install matplotlib numpy scipy

!pip install ipywidgets
#To add buttons in our output for user to verify detected noises

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.5 MB/s eta 0:00:00


In [ ]:
# Import Libraries
import glob #look into this. A way to handle directories in python
import numpy as np
from scipy.io import wavfile
from scipy.io.wavfile import write
import noisereduce as nr
import auditok
import pandas as pd
import datetime
import os
import statistics

import ipywidgets as widgets
from ipywidgets import Button, Output, IntSlider
from IPython.display import display, clear_output, Audio, HTML  # Import Audio and display for playback


import matplotlib.pyplot as plt

import asyncio


# Mounting Google Drive and finding folders
The Google Drive has all the audio files we will need to look at
**sure you had starred the "Cat Song Meter Recordings" google drive to "My Drive"**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


**Below you will put the path to the folder you want to look at that contains 1 hour incremented audio files. Put your desired path in the 2nd line of code, 'base_dir' variable.**

Google Drive Tree:

    \content
      \drive
        \MyDrive
          \Cat Song Meter Recordings
            \ Desired Folder...

In [ ]:
#Directory paths- insert folder path in 'base_dir'
base_dir = '/content/drive/MyDrive/Cat Song Meter Recordings/Capstone Code Review Folder' # Edit this
reduced_audio_dir = os.path.join(base_dir, 'ReducedAudio')  # Folder that will hold reduced audio files
saw_calls_dir = os.path.join(base_dir, 'DetectedSawCalls')  # Folder that will hold detected saw calls

#Ensure directories exist
os.makedirs(reduced_audio_dir, exist_ok=True)
os.makedirs(saw_calls_dir, exist_ok=True)

#Extracts path for .wav files
file_list = glob.glob(os.path.join(base_dir, '*.wav')) # extract all files with .wav
    # The '*' prints keep everything before the *
    # so it keeps up to '/content/drive/MyDrive/Cat Song Meter Recordings/...    when extracting a file

#Print all the files in that file_list
file_list

['/content/drive/MyDrive/Cat Song Meter Recordings/Capstone Code Review Folder/SMM07257_20221117_153302.wav']

In [ ]:
for file in file_list:
  filename = file[-28:-4] #slicing- removing all the non-important details from the file name
  print("File Name that will be Noise Reduced and Detect Noises: " + filename)

File Name that will be Noise Reduced and Detect Noises: SMM07257_20221117_153302


# Noise Reduction:
Noise reduction is crucial for accurate sound detection, which is a core requirement in identifying the vocalizations of big cats. This code will take a 1-hour long RAW audio file and reduce the noise in it, creating a new reduces noise audio file.

Background: Weekly, the backend will process daily recordings, which consist of 16 one-hour incremented raw audio files (recorded between 4 pm and 8 am), reducing background noise and creating 16 new noise-reduced audio files.

In [ ]:
def noise_reduction(file, output_dir):
    """ Reduces background noise from audio files and saves them."""
    rate, data = wavfile.read(file)  # Read the .wav file

    # Define new file name for the reduced file as filename_reduced.wav
    filename = os.path.basename(file)[:-4] + "_reduced.wav"

    # Check if the reduced file already exists
    reduced_file_path = os.path.join(output_dir, filename)
    if os.path.exists(reduced_file_path):
        # Prompt the user for action
        user_input = input(f"Reduced file already exists: {reduced_file_path}. Do you want to overwrite it? (y/n): ").strip().lower()
        if user_input != 'y':
            print(f"Keeping existing reduced file: {reduced_file_path}")
            return  # Exit the function if the user chooses not to overwrite
    print("Overwritting the existing file and creating a new reduced file is in process")

    # Perform noise reduction
    reduced_noise = nr.reduce_noise(y=data, sr=rate)

    # Save the reduced audio to the output directory
    write(reduced_file_path, rate, reduced_noise)
    print(f"Noise reduction complete for {file}")

In [ ]:
# Iterate over all original .wav audio files and perform noise reduction(eliminating background noise) on each
for file in file_list:
    noise_reduction(file, reduced_audio_dir)

<ipython-input-8-e3485cff2dc1>:3: WavFileWarning: Chunk (non-data) not understood, skipping it.
  rate, data = wavfile.read(file)  # Read the .wav file


Reduced file already exists: /content/drive/MyDrive/Cat Song Meter Recordings/Capstone Code Review Folder/ReducedAudio/SMM07257_20221117_153302_reduced.wav. Do you want to overwrite it? (y/n): n
Keeping existing reduced file: /content/drive/MyDrive/Cat Song Meter Recordings/Capstone Code Review Folder/ReducedAudio/SMM07257_20221117_153302_reduced.wav


In [ ]:
# This extracts the path way for each reduced noise .wav file and prints it to user
audio_list = glob.glob(os.path.join(reduced_audio_dir, '*_reduced.wav'))
print(f"File path for the new reduced noise audio file: {audio_list}")
print("Reduced Audio Directory Contents:", os.listdir(reduced_audio_dir))

File path for the new reduced noise audio file: ['/content/drive/MyDrive/Cat Song Meter Recordings/Capstone Code Review Folder/ReducedAudio/SMM07257_20221117_153302_reduced.wav']
Reduced Audio Directory Contents: ['SMM07257_20221117_153302_reduced.wav']


# Plotting Waveforms and Spectrograms

In [ ]:
# This function creates a waveform and spectrogram for each detected noise file and displays it
def plot_waveform_and_spectrogram(file, start_time, end_time, reduced_file):
    # Read the audio file
    rate, data = wavfile.read(file)

    # Create time axis for waveform
    time = np.linspace(0, len(data) / rate, num=len(data))

    # Create figure for waveform and spectrogram
    fig, axs = plt.subplots(2, 1, figsize=(12, 8))

    # Plot waveform
    axs[0].plot(time, data)
    axs[0].set_title(f'Waveform\n Reduced File: {reduced_file[-28:-4]}   Detected File: {file[-28:-4]},\n Region: {start_time} - {end_time}')
    axs[0].set_xlabel('Time (s)')
    axs[0].set_ylabel('Amplitude')

    # Plot spectrogram
    axs[1].specgram(data, Fs=rate, NFFT=1024, noverlap=512, cmap='viridis')
    axs[1].set_title(f'Spectrogram\n Reduced File: {reduced_file[-28:-4]}   Detected File: {file[-28:-4]},\n Region: {start_time} - {end_time}')
    axs[1].set_xlabel('Time (s)')
    axs[1].set_ylabel('Frequency (Hz)')

    # Add space between plots
    plt.subplots_adjust(hspace=1.5)  # Increase hspace to add space between the two plots
    plt.tight_layout()
    plt.show()

  #Just prints spectrogtram
def plot_onlyspectrogram(file, start_time, end_time, reduced_file):
    # Read the audio file
    rate, data = wavfile.read(file)

    # Create time axis for waveform
    time = np.linspace(0, len(data) / rate, num=len(data))

    # Create figure for the plot
    plt.figure(figsize=(10, 6))

    # Plot spectrogram
    Pxx, freqs, bins, im = plt.specgram(data, Fs=rate, NFFT=1024, noverlap=512, cmap='gray')  # Adjusted to gray colormap

    # Convert frequency axis to kHz for clearer presentation
    plt.ylim(0, 12000)  # Show frequencies up to 12kHz (matching your example)
    plt.yticks(np.arange(0, 13000, 1000), labels=[f'{i/1000:.0f}' for i in np.arange(0, 13000, 1000)])  # kHz labels

    # Title including the file name and time range
    plt.title(f'Spectrogram\n Reduced File: {reduced_file[-28:-4]}    Detected File: {file[-28:-4]},\n Region: {start_time} - {end_time}', pad=10)
    plt.xlabel('Time (mm:ss)')
    plt.ylabel('Frequency (kHz)')

    # Format x-axis labels to show time in mm:ss
    x_ticks = plt.gca().get_xticks()
    plt.gca().set_xticklabels([f'{int(t//60)}:{int(t%60):02d}' for t in x_ticks])

    plt.tight_layout() # Adjust layout to prevent overlap

    plt.show() # Show the plot

# Classifications through buttons

In [ ]:
# Function to handle the classification and counting of sounds
def classify_sound_Old(detected_file_path, dataset, filename, region_index, processing_timestamp):
    # Create an Output widget to capture print statements
    output = Output()

    # Display buttons for user classification
    button_saw = Button(description="SAW")
    button_neither = Button(description="Neither")
    button_calls = Button(description="CALLS")

    # Create a slider to input counts
    slider_saws = IntSlider(value=0, min=0, max=10, description='Number of SAWS:', style={'description_width': 'initial'})
    slider_calls = IntSlider(value=0, min=0, max=10, description='Number of CALLS:', style={'description_width': 'initial'})

    # Clear output and show the slider based on button clicks
    def clear_and_display_slider(slider):
        with output:
            clear_output(wait=True)
            display(button_saw, button_neither, button_calls)
            display(slider)

    # Define callback functions for button clicks
    def on_saw_clicked(b):
        clear_and_display_slider(slider_saws)

    def on_neither_clicked(b):
        with output:
            clear_output(wait=True)
            dataset.append([filename[:-8], region_index, "Neither", 0, processing_timestamp])
            print(f"Recorded 'Neither' for {filename[:-8]}, region {region_index}.")

    def on_calls_clicked(b):
        clear_and_display_slider(slider_calls)
        clear_and_display_slider(slider_saws)

    # Define what happens when the slider changes
    def on_saw_slider_change(change):
        if change['new'] > 0:
            dataset.append([filename[:-8], region_index, "SAW", change['new'], processing_timestamp])
            with output:
                clear_output(wait=True)
                print(f"Recorded {change['new']} SAW(s) for {filename[:-8]}, region {region_index}.")
                display(button_saw, button_neither, button_calls)

    def on_calls_slider_change(change):
        if change['new'] > 0:
            dataset.append([filename[:-8], region_index, "CALLS", change['new'], processing_timestamp])
            with output:
                clear_output(wait=True)
                print(f"Recorded {change['new']} CALL(S) for {filename[:-8]}, region {region_index}.")
                display(button_saw, button_neither, button_calls)

    # Assign callback functions to button clicks
    button_saw.on_click(on_saw_clicked)
    button_neither.on_click(on_neither_clicked)
    button_calls.on_click(on_calls_clicked)

    # Assign slider change functions
    slider_saws.observe(on_saw_slider_change, names='value')
    slider_calls.observe(on_calls_slider_change, names='value')

    # Display buttons initially
    display(button_saw, button_neither, button_calls)
    display(output)

# Sample usage of classify_sound
# classify_sound("path/to/detected/file.wav", dataset, "filename.wav", region_index, "timestamp")


In [ ]:
# Create a Future that will be set once the user has submitted their input
user_input_future = None

# Function to handle the classification and counting of sounds using text input widget
async def classify_sound(detected_file_path, dataset, filename, region_index, processing_timestamp):
  global user_input_future  # Use global so it can be set from within the inner function
  user_input_future = asyncio.Future()  # Reset future for each classification

  # Create buttons for user classification
  button_saw = widgets.Button(description="Just SAWs")
  button_neither = widgets.Button(description="Neither")
  button_calls = widgets.Button(description="CALLs & SAWs ")

  # Create an output widget to capture user interactions
  output = widgets.Output()

  # Create a text input widget for manual input of counts
  input_box = widgets.Text(description=f'Enter count:', placeholder='Enter number')
  submit_button = widgets.Button(description="Submit Count")  # Define the submit button

  # Function to clear and display text input for user to enter SAW and/or CALL counts
  async def clear_and_ask_input(sound_type):
      with output:
          clear_output(wait=True)
          print(f"How many {sound_type} were detected?")
          display(input_box, submit_button)  # Display input box and submit button
          input_box.value = ''  # Clear previous input if any

          # Function triggered when user presses Enter
          def on_submit(change):
              try:
                  count = int(input_box.value)  # Get input from the text box
                  if sound_type == "SAW":
                      dataset[region_index][7] = "SAW"   # Sound Type
                      dataset[region_index][8] = count   # Total SAWS
                  elif sound_type == "CALLS":
                      dataset[region_index][7] = "CALLS" # Sound Type
                      dataset[region_index][9] = count   # Total CALLS

                  clear_output(wait=True)
                  print(f"Recorded {count} {sound_type}(s) for {filename[:-8]}, region {region_index}.")
              except ValueError:
                  clear_output(wait=True)
                  print(f"Invalid input. Please enter a valid number for {sound_type}.")
                  display(input_box)

          input_box.on_submit(on_submit)  # Triggered when user presses Enter
          await asyncio.sleep(.5)  # Wait for user interaction

  # Button click actions  # If user clicked the SAW button
  def on_saw_clicked(b):
    asyncio.ensure_future(clear_and_ask_input("SAW"))

  def on_neither_clicked(b): #If user clicked the neither button
    with output:
        clear_output(wait=True)
        dataset[region_index][7] = "Neither"
        dataset[region_index][8] = 0  # Total SAWS
        dataset[region_index][9] = 0  # Total CALLS
        print(f"Recorded 'Neither' for {filename[:-8]}, region {region_index}.")
        user_input_future.set_result(None)  # Set the future to indicate input is done #new


  def on_calls_clicked(b):
    asyncio.ensure_future(clear_and_ask_input("CALLS"))
    asyncio.ensure_future(clear_and_ask_input("SAW"))

  submit_button.on_click(lambda b: user_input_future.set_result(None))  # When user submits count


  # Assign button click functions
  button_saw.on_click(on_saw_clicked)
  button_neither.on_click(on_neither_clicked)
  button_calls.on_click(on_calls_clicked)

  # Display buttons and output
  display(button_saw, button_neither, button_calls)
  display(output)


# Detected Noise Extraction
Detection of Big Cat Sounds: The backend will analyze the noise-reduced audio files and extract sections where the energy exceeds a certain threshold. These snippets, representing potential Big Cat vocalizations (primarily SAW CALLS), are saved as new files. However, It does sometimes detect other sounds similar to it and extracts them, we would need a user to double check these later.

In [ ]:
# Function to save the dataset to an Excel file
# Example dataset: [['file1', 1, '00:00', '00:05', '.5 seconds', 'August 1st', 'SAW', '1', '0']
def save_to_excel(dataset, base_dir):
  # Convert the dataset to a pandas DataFrame
  df = pd.DataFrame(dataset, columns=[
      'File', 'Detected File', 'Region', 'Start Time', 'End Time',
      'Duration', 'Processing Timestamp', 'Sound Type',
      'Total SAWS', 'Total CALLS'
  ])

  # Define the output path for the Excel file
  output_file_path = os.path.join(base_dir, 'output.xlsx')

  # Save the DataFrame to the Excel file (overwriting if it exists)
  df.to_excel(output_file_path, index=False)

  print(f"Excel file saved at {output_file_path}")







**This Function makes the output interactive allowing the user to verify if the detected noise is a SAW, CALL, or Neither**

In [ ]:
# Function to check if the saw_calls_dir (DetectedSawCalls Folder) already contains detected files and report the count
def check_and_prompt_for_deletion(saw_calls_dir):
  # Check if the directory exists and contains any files
  if os.path.exists(saw_calls_dir):
    files_in_dir = os.listdir(saw_calls_dir)
    file_count = len([file for file in files_in_dir if os.path.isfile(os.path.join(saw_calls_dir, file))])  # Count only files

    if file_count > 0:
        print(f"{file_count} detected files found in {saw_calls_dir}.")
        user_input = input("Do you want to delete them and rerun detection? (yes/no): ")

        if user_input.lower() == 'yes':
            # Delete all files in the directory
            for file in files_in_dir:
                file_path = os.path.join(saw_calls_dir, file)
                try:
                    if os.path.isfile(file_path):
                        os.remove(file_path)
                except Exception as e:
                    print(f"Error deleting file {file}: {e}")
            print(f"All {file_count} old detected files deleted from {saw_calls_dir}.")
            return True  # Proceed with the detection phase
        else:
            print("Skipping the detection phase.")
            return False  # Skip the detection phase
  return True  # If no files exist, proceed with the detection phase

# For each .wav file, this function extracts regions of interest (audio segments where noise exceeds a given threshold) and saves them as audio clips in a specified directory.
async def audio_pipeline(file, dataset, saw_calls_dir):
  # Check if the user wants to delete old files before starting detection
  if not check_and_prompt_for_deletion(saw_calls_dir):
      return  # If the user doesn't want to rerun detection, exit this function

  #filename = file[5:-4] # it will call the file like this SMM10131_20240320_143902_reduced
  # Extracts the filename without the directory path and file extension so from path/to/file/SMM10131_20240320_143902_reduced.wav to SMM10131_20240320_143902_reduced
  filename = os.path.basename(file)[:-4]

  # Check if the input file(reduced audio file) exists
  if not os.path.exists(file):
      print(f"Reduced File not found: {file}")
      return

  # Get the current processing timestamp for debugging purposes
  processing_timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

  # Use auditok to split the audio into regions where the sound reaches a certain energy threshold (hears a possible SAW CALL noise)
  audio_regions = auditok.split(file,
                                min_dur=0.2,        # minimum duration of a valid audio event (.2 seconds)
                                max_dur=3.5,        # maximum duration of a detected sound region (.5 seconds)
                                max_silence=0.3,    # Maximum allowed silence within a sound region (0.3 seconds)
                                energy_threshold=60 # The energy level that qualifies as a sound to be detected (60)
                                )

  # If no audio regions were detected, print a message and return.
  if not audio_regions:
      print(f"No audio regions detected in {file}. Please, check the energy threshold or file content.")
      return

  # Initialize variables to track call counts in 5-second intervals
  average_n_calls = 0    # Tracks the average number of calls
  curr = 5               # Tracks the average number of calls
  curr_count = 0         # Counts how many calls are detected in the current 5-second window
  count_list = []        # Stores counts of calls detected in each 5-second window

  # Loop through all the detected audio regions
  for i, r in enumerate(audio_regions):
    #print("Region {i}: {r.meta.start:.3f}s -- {r.meta.end:.3f}s".format(i=i, r=r))
    region = i
    start = str(datetime.timedelta(seconds=r.meta.start))
    end = str(datetime.timedelta(seconds=r.meta.end))

    # Calculate the duration of the region
    dur = r.meta.end - r.meta.start

    # Format duration to include ' seconds' in output spreadsheet
    formatted_dur = f"{dur:.2f} seconds"  # Convert duration to string with ' seconds'

    # Skip regions shorter than 0.04 seconds (to filter out very short noise events)
    if dur < 0.04:
      print(f"Skipped, duration = {dur}")
      continue

    # Update the call count for the current 5-second interval
    if r.meta.start < curr:
      curr_count += 1
    else:
      curr += 5   # Move to the next 5-second interval
      count_list.append(curr_count)  # Store the count for the previous interval
      curr_count = 1    # Start a new count for the new interval

    # Save detected noise region as a .wav audio file in the 'DetectedSawCalls' folder
    detected_file_path = os.path.join(saw_calls_dir, f"region_{r.meta.start:.3f}-{r.meta.end:.3f}.wav")
    r.save(detected_file_path)

    # Storing information about the detected region in the dataset for further analysis shown in the output.xlsx
    dataset.append([
        filename[:-8], # Base Reduced filename
        detected_file_path[-28:-4], # Detected filename
        i,             # Region index
        start[:-4],    # Start time
        end[:-4],      # End time
        formatted_dur,         # Duration of the detected sound
        processing_timestamp,  # Timestamp of processing
        None,          # Placeholder for sound type (SAW or CALLS)
        0,             # Placeholder for number of SAWS detected
        0              # Placeholder for number of CALLS detected
        ])
        # filename[:-8]: Removes the last 8 characters from the filename(typically the "_reduced" suffix)


    # Display the audio player with the detected audio file name centered
    display(HTML(f"""
        <div style="text-align: center;">
            <h3>Detected File: {detected_file_path[-28:-4]}</h3>
            {Audio(detected_file_path)._repr_html_()}
        </div>
    """))
    #display(Audio(detected_file_path))  # Display the audio player for playback

    # Plot the waveform and spectrogram with file name and time range detected
    '''plot_waveform_and_spectrogram(detected_file_path, start, end, file)'''

    # Create buttons for classification
    button_saw = widgets.Button(description="SAW")
    button_neither = widgets.Button(description="Neither")
    button_call = widgets.Button(description="CALL")
    output = widgets.Output()

    # Define button click functions
    def on_saw_clicked(b):
        with output:
            dataset[-1][7] = "SAW"
            dataset[-1][8] += 1  # Increment SAW count
            print("Classified as SAW")
        clear_output(wait=True)
        save_to_excel(dataset, base_dir)  # Automatically save the updated dataset

    def on_neither_clicked(b):
        with output:
            dataset[-1][7] = "Neither"
            print("Classified as Neither")
        clear_output(wait=True)
        save_to_excel(dataset, base_dir)  # Automatically save the updated dataset

    def on_call_clicked(b):
        with output:
            dataset[-1][7] = "CALL"
            dataset[-1][9] += 1  # Increment CALL count
            print("Classified as CALL")
        clear_output(wait=True)
        save_to_excel(dataset, base_dir)  # Automatically save the updated dataset

    # Assign the buttons to the event handlers
    button_saw.on_click(on_saw_clicked)
    button_neither.on_click(on_neither_clicked)
    button_call.on_click(on_call_clicked)

    # Display buttons and output
    display(button_saw, button_neither, button_call, output)

    # Await user input and proceed
    while dataset[-1][7] is None:
        await asyncio.sleep(0.1)  # Small pause to allow for button interaction

    # Clear output after each classification
    clear_output(wait=True)

    # Call the classify_sound function for user to varify the detected noise
    '''await classify_sound(detected_file_path, dataset, filename, i, processing_timestamp)'''
    #classify_sound_Old(detected_file_path, dataset, filename, i, processing_timestamp)

  # Once all regions have been processed, print a summary
  print(f"File processed: {filename[:-8]}")

  # If call counts were recorded, print the average number of calls per 5 seconds and the total number of calls
  if count_list:
    print(f"Avg/5sec: {statistics.fmean(count_list)}")
    print(f"Total number of Detected Saw Calls: {sum(count_list)}")
  else:
    print("No calls detected in the current 5-second intervals.")
  print(" ")

In [ ]:
dataset = [] #This holds all the data about the detected noise that will be placed in a excel file

# iterating over all noise reduced files and applying the above function on each
for file in audio_list:
  await audio_pipeline(file, dataset, saw_calls_dir)

1 detected files found in /content/drive/MyDrive/Cat Song Meter Recordings/Capstone Code Review Folder/DetectedSawCalls.
Do you want to delete them and rerun detection? (yes/no): yes
All 1 old detected files deleted from /content/drive/MyDrive/Cat Song Meter Recordings/Capstone Code Review Folder/DetectedSawCalls.


AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import os
import pandas as pd
from datetime import datetime

# Example dataset with columns
dataset = [['file1', 1, '00:00', '00:05', '5 sec', '', '', 0, 0],
           ['file2', 2, '00:05', '00:10', '5 sec', '', '', 0, 0]]

base_dir = './'  # Adjust this to your actual base directory for saving

# Function to save the Excel file after classification
def save_to_excel(dataset, base_dir):
    df = pd.DataFrame(dataset, columns=['File', 'Detected File', 'Region', 'Start Time', 'End Time', 'Duration',
                                        'Processing Timestamp', 'Sound Type', 'Total SAWS', 'Total CALLS'])
    output_file_path = os.path.join(base_dir, 'output.xlsx')
    df.to_excel(output_file_path, index=False)
    print(f"Excel file saved at {output_file_path}")

# Function to display buttons and classify the sound type
def display_buttons_and_get_classification(dataset, base_dir):
    # Initialize the buttons
    button_saw = widgets.Button(description="SAW")
    button_neither = widgets.Button(description="Neither")
    button_call = widgets.Button(description="CALL")

    # Define button click functions
    def on_saw_clicked(b):
        print("SAW button clicked")
        dataset[-1][7] = "SAW"
        dataset[-1][8] += 1  # Increment the count of SAWS
        save_to_excel(dataset, base_dir)

    def on_neither_clicked(b):
        print("Neither button clicked")
        dataset[-1][7] = "Neither"
        save_to_excel(dataset, base_dir)

    def on_call_clicked(b):
      print(f"CALL button clicked")
      print(f"Current dataset row: {dataset[-1]}")  # Print current row for debugging
      print("CALL button clicked")
      dataset[-1][7] = "CALL"
      dataset[-1][9] += 1  # Increment the count of CALLS
      save_to_excel(dataset, base_dir)


    # Ensure the row has enough elements
    if len(dataset[-1]) < 10:
        print(f"Error: dataset row does not have enough elements. Current length: {len(dataset[-1])}")
        return

    dataset[-1][7] = "CALL"
    dataset[-1][9] += 1  # Increment the count of CALLS
    save_to_excel(dataset, base_dir)


    # Assign button click events
    button_saw.on_click(on_saw_clicked)
    button_neither.on_click(on_neither_clicked)
    button_call.on_click(on_call_clicked)

    # Display buttons
    display(button_saw, button_neither, button_call)

# Run the function to test
display_buttons_and_get_classification(dataset, base_dir)


Error: dataset row does not have enough elements. Current length: 9


# Results Through Excel

In [ ]:
from IPython.display import display, Audio, HTML
import ipywidgets as widgets
import pandas as pd
import os
from scipy.io import wavfile

# Load the dataset from the Excel file
def load_dataset(output_file_path):
    return pd.read_excel(output_file_path)

# Function to format time for file naming
def format_time_for_filename(time_str):
    # Convert "0:27:17.95" to "1627.950"
    time_parts = time_str.split(':')
    minutes = int(time_parts[0])
    seconds = float(time_parts[1]) + (minutes * 60)
    return f"{seconds:.3f}"

# Function to display and classify each detected noise
def review_and_classify_noises(dataset, detected_sounds_dir, output_file_path):

    for index, row in dataset.iterrows():
        if pd.isnull(row['Sound Type']):  # Only review unclassified sounds
            # Ensure Start Time and End Time are formatted as floats
            start_time = float(row['Start Time'].split(':')[2]) + float(row['Start Time'].split(':')[1]) * 60 + float(row['Start Time'].split(':')[0]) * 3600
            end_time = float(row['End Time'].split(':')[2]) + float(row['End Time'].split(':')[1]) * 60 + float(row['End Time'].split(':')[0]) * 3600

            detected_file_path = os.path.join(detected_sounds_dir, f"region_{start_time:.3f}-{end_time:.3f}.wav")



    def display_noise(index):
      if index < len(dataset) and pd.isnull(dataset.at[index, 'Sound Type']):
          # Ensure Start Time and End Time are formatted as floats
          start_time = format_time_for_filename(dataset.at[index, 'Start Time'])
          end_time = format_time_for_filename(dataset.at[index, 'End Time'])

          detected_file_path = os.path.join(detected_sounds_dir, f"region_{start_time}-{end_time}.wav")

          # Check if the detected file exists before displaying
          if os.path.exists(detected_file_path):
              # Load the audio file to get the sampling rate
              rate, _ = wavfile.read(detected_file_path)

              # Display the detected audio
              display(HTML(f"""
                  <div style="text-align: center;">
                      <h3>Detected File: {detected_file_path[-28:-4]}</h3>
                      {Audio(detected_file_path, rate=rate)._repr_html_()}
                  </div>
              """))
              # Here, you would call your plot_waveform_and_spectrogram function as needed
              # Check if the detected file exists before displaying
              if os.path.exists(detected_file_path):
                  # Load the audio file to get the sampling rate
                  rate, _ = wavfile.read(detected_file_path)

                  # Display the detected audio, waveform, and spectrogram
                  display(HTML(f"""
                      <div style="text-align: center;">
                          <h3>Detected File: {detected_file_path[-28:-4]}</h3>
                          {Audio(detected_file_path, rate=rate)._repr_html_()}
                      </div>
                  """))
                  plot_waveform_and_spectrogram(detected_file_path, row['Start Time'], row['End Time'], row['File'])

              # Create classification buttons
              button_saw = widgets.Button(description="SAW")
              button_neither = widgets.Button(description="Neither")
              button_calls = widgets.Button(description="CALLS")
              input_box = widgets.IntText(description='Count:', placeholder='Enter number', value=0)
              submit_button = widgets.Button(description="Submit")
              output = widgets.Output()

              classification = None  # To hold the selected classification

              def on_saw_clicked(b):
                  nonlocal classification
                  classification = "SAW"
                  output.clear_output()
                  with output:
                      print("You selected: SAW")

              def on_neither_clicked(b):
                  nonlocal classification
                  classification = "Neither"
                  output.clear_output()
                  with output:
                      print("You selected: Neither")
                  input_box.value = 0  # Reset count for 'Neither'

              def on_calls_clicked(b):
                  nonlocal classification
                  classification = "CALLS"
                  output.clear_output()
                  with output:
                      print("You selected: CALLS")

              button_saw.on_click(on_saw_clicked)
              button_neither.on_click(on_neither_clicked)
              button_calls.on_click(on_calls_clicked)

              def on_submit_clicked(b):
                  if classification is not None:
                      count = input_box.value
                      update_dataset(index, classification, count)
                      next_noise()
                  else:
                      print("Please select a classification before submitting.")

              submit_button.on_click(on_submit_clicked)

              display(button_saw, button_neither, button_calls, input_box, submit_button, output)
          else:
              print(f"File not found: {detected_file_path}")
              next_noise()  # Move to the next noise if the file doesn't exist
      else:
          print("No more unclassified noises.")

    def next_noise():
        nonlocal index
        index += 1
        # Clear previous output
        clear_output(wait=True)  # Clear previous output
        display_noise(index)


    # Start displaying the first noise
    display_noise(index)

# Function to update the dataset with classification and counts
def update_dataset(index, sound_type, count):
    dataset.at[index, 'Sound Type'] = sound_type
    if sound_type == "SAW":
        dataset.at[index, 'Total SAWS'] = int(count)
    elif sound_type == "CALLS":
        dataset.at[index, 'Total CALLS'] = int(count)

    # Save the updated dataset back to the Excel file
    dataset.to_excel(output_file_path, index=False)
    print(f"Updated: {sound_type} with {count}")

# Load the dataset and review noises
output_file_path = '/content/drive/MyDrive/Cat Song Meter Recordings (1)/Capstone Code Review Folder/output.xlsx'  # Replace with your output file path
detected_sounds_dir = '/content/drive/MyDrive/Cat Song Meter Recordings (1)/Capstone Code Review Folder/DetectedSawCalls'  # Replace with your detected sounds directory
dataset = load_dataset(output_file_path)

# Print dataset and unclassified noises count for debugging
print(dataset)
unclassified_count = dataset['Sound Type'].isnull().sum()
print(f"Unclassified noises count: {unclassified_count}")

# Convert the dataset to a DataFrame (assuming dataset is a list of lists)
df = pd.DataFrame(dataset, columns=['File', 'Region', 'Start Time', 'End Time', 'Duration', 'Processing Timestamp', 'Sound Type', 'Total SAWS', 'Total CALLS'])

# Then pass the DataFrame instead of the list
review_and_classify_noises(df, detected_sounds_dir, output_file_path)


Empty DataFrame
Columns: [File, Detected File, Region, Start Time, End Time, Duration, Processing Timestamp, Sound Type, Total SAWS, Total CALLS]
Index: []
Unclassified noises count: 0


UnboundLocalError: local variable 'index' referenced before assignment

In [ ]:
unclassified_count = dataset['Sound Type'].isnull().sum()
print(f"Unclassified noises count: {unclassified_count}")

print(dataset.columns)

print(df.shape)
print(df.columns)



Unclassified noises count: 7
Index(['File', 'Region', 'Start Time', 'End Time', 'Duration',
       'Processing Timestamp', 'Sound Type', 'Total SAWS', 'Total CALLS'],
      dtype='object')
(7, 9)
Index(['File', 'Region', 'Start Time', 'End Time', 'Duration',
       'Processing Timestamp', 'Sound Type', 'Total SAWS', 'Total CALLS'],
      dtype='object')


In [ ]:
#Habibul's notes
# display visuals in the website. Here's more information on it.  https://github.com/amsehili/auditok
#etl process extract transform load
#maybe theirs a google drive api to easly connect it
#download audacity to see how the audio clip would look like.

In [ ]:
# Habibul's extra helpful code for testing purposes:

# Prints all the saved files in a specific folder
file_list = glob.glob('/content/drive/MyDrive/Cat Song Meter Recordings/ReducedAudio/*')
print(file_list)

#To save the new audio files into a different folder
destination_path = '/content/drive/MyDrive/Cat Song Meter Recordings/ReducedAudio/output.txt'

# Move the file to the specified directory in Google Drive
!cp output.txt "$destination_path"


''' If you want to hear a specific audio for testing purposes'''
from IPython.display import Audio

# Specify the path to your audio file
audio_file_path = '/content/drive/MyDrive/Cat Song Meter Recordings (1)/Capstone Code Review Folder/DetectedSawCalls/region_1637.950-1638.500.wav'

# Create an audio player
audio_player = Audio(audio_file_path)

# Play the audio
audio_player



#Habibul's Test Code to upload reduced audio and detected audio to the drive

In [ ]:
# Directory paths
base_dir = '/content/drive/MyDrive/Cat Song Meter Recordings (1)/Amur Leopard/Dec 15 - Dec 21 7257 inside'
reduced_audio_dir = os.path.join(base_dir, 'ReducedAudio')  # Folder for reduced audio
saw_calls_dir = os.path.join(base_dir, 'DetectedSawCalls')  # Folder for detected saw calls


#reduced_audio_dir = f'{base_dir}/ReducedAudio'  # Folder for reduced audio
#saw_calls_dir = f'{base_dir}/DetectedSawCalls'  # Folder for detected saw calls

# Ensure directories exist
os.makedirs(reduced_audio_dir, exist_ok=True)
os.makedirs(saw_calls_dir, exist_ok=True)

#extracts path for .wav files
#file_list = glob.glob('/content/drive/MyDrive/Cat Song Meter Recordings/Amur Leopard/Dec 15-Dec 21 7257 inside/*.wav')#Hammadas #extract all files with wav
#file_list = glob.glob(f'{base_dir}/*.wav') #extract all files with wav
file_list = glob.glob(os.path.join(base_dir, '*.wav'))
    #* prints keep everything before the *
    # keep up to '/content/drive/MyDrive/Cat Song Meter Recordings/...    when extrating a file

#print all the files in that file_list
  #file_list


#extracts path for .wav files that have no background noise
#audio_list = glob.glob('data/*_reduced.wav') #Hammad's code
audio_list = glob.glob(os.path.join(reduced_audio_dir, '*_reduced.wav'))
#audio_list = glob.glob(f'{reduced_audio_dir}/*_reduced.wav') #Habibul's code
print(f"File path for new reduced noise audio: {audio_list}")


#Noise Reduction Phase

# iterates over .wav files and eliminates background noise
# saves modified audio in the following format --> filename_reduced.wav
for file in file_list:
    rate, data = wavfile.read(file)
    filename = os.path.basename(file)[:-4]
    #directory = "data/"   #all noise reduced files goes into data directory
    # perform noise reduction
    reduced_noise = nr.reduce_noise(y=data, sr=rate)

    #new_name = filename + "_reduced.wav" #Hammad's code
    #write(directory+new_name, rate, reduced_noise) #Hammad's codev

    new_name = f"{filename}_reduced.wav"
    write(os.path.join(reduced_audio_dir, new_name), rate, reduced_noise)  # Save reduced audio to the reduced_audio_dir

    #new_name = f'{reduced_audio_dir}/{filename}_reduced.wav' #Habibul's code
    #write(new_name, rate, reduced_noise) #Habibul's code
print("Noise reduction is complete.")


#extracts path for .wav files that have no background noise
#audio_list = glob.glob('data/*_reduced.wav') #Hammad's code
audio_list = glob.glob(os.path.join(reduced_audio_dir, '*_reduced.wav'))
#audio_list = glob.glob(f'{reduced_audio_dir}/*_reduced.wav') #Habibul's code
print(f"File path for new reduced noise audio: {audio_list}")



#Detected Noise Extraction Phase

# for each .wav file, extracts regions of interest and saves them as clips
def audio_pipeline(file, dataset):
    #filename = file[5:-4] # it will call the file like this SMM10131_20240320_143902_reduced
    filename = os.path.basename(file)[:-4]
    #os.mkdir("data//"+filename)
    #os.makedirs("data//"+filename,exist_ok=True) #Hammad's code #creating new folder with the same name under the data file

    #Create a folder for detected regions under saw calls folder
    #detected_folder = f'{saw_calls_dir}/{filename}'
    #os.makedirs(detected_folder, exist_ok=True)

    audio_regions = auditok.split(file,
                                  min_dur=0.2,        # minimum duration of a valid audio event in seconds
                                  max_dur=0.5,        # maximum duration of an event
                                  max_silence=0.3,    # maximum duration of tolerated continuous silence within an event
                                  energy_threshold=60 # threshold of detection
                                 )
    average_n_calls = 0
    curr = 5
    curr_count = 0
    count_list = []

    for i, r in enumerate(audio_regions):
        #print("Region {i}: {r.meta.start:.3f}s -- {r.meta.end:.3f}s".format(i=i, r=r))
        region = i
        start = str(datetime.timedelta(seconds=r.meta.start))
        end = str(datetime.timedelta(seconds=r.meta.end))

        #

        dur = r.meta.end - r.meta.start
        if dur < 0.04:
          print(f"Skipped, duration = {dur}")
          continue

        #

        if r.meta.start < curr:
          curr_count += 1
        else:
          curr += 5
          count_list.append(curr_count)
          curr_count = 1

        #

        dataset.append([filename[:-8], i, start[:-4], end[:-4]])
        #r.save("data//" + filename + "//region_{meta.start:.3f}-{meta.end:.3f}.wav")  #Hammad's code

        # Save detected region as .wav in detected saw calls folder
        #r.save(f'{detected_folder}/region_{r.meta.start:.3f}-{r.meta.end:.3f}.wav')
        r.save(os.path.join(saw_calls_dir, f"region_{r.meta.start:.3f}-{r.meta.end:.3f}.wav")) #Habibul's


    print(f"{filename[:-8]}")
    print(f"Avg/5sec : {statistics.fmean(count_list)}")
    print(f"Total no. of Detected Saw Calls: {sum(count_list)}")
    print(" ")


dataset = []
# iterating over all noise reduced files and applying the above function on each
for file in audio_list:
    audio_pipeline(file, dataset)

NotADirectoryError: [Errno 20] Not a directory: '/content/drive/MyDrive/Cat Song Meter Recordings (1)/Amur Leopard/Dec 15 - Dec 21 7257 inside/SMM07257_20221215_133531.wav/ReducedAudio'

In [ ]:
import matplotlib.pyplot as plt

# Function to visualize the waveform of an audio file
def plot_waveform(file_path):
    rate, data = wavfile.read(file_path)

    # Plot the waveform
    plt.figure(figsize=(14, 5))
    plt.plot(data)
    plt.title(f'Waveform of {os.path.basename(file_path)}')
    plt.xlabel('Sample Number')
    plt.ylabel('Amplitude')
    plt.xlim(0, len(data))  # Limit x-axis to the length of the audio data
    plt.show()

# Call the function to visualize one of the extracted noises
if audio_list:  # Check if audio_list is not empty
    plot_waveform(audio_list[0])  # Visualize the first reduced noise file
else:
    print("No reduced noise files found.")


No reduced noise files found.


Use the code below to display waveform and spectogram of a single audio file.

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

def display_waveform_and_spectrogram(audio_path):
    # Load audio file with downsampling for faster processing
    y, sr = librosa.load(audio_path, sr=22050)  # Downsampled to 22.05 kHz

    # Optionally select a shorter segment of the audio (e.g., first 30 seconds)
    y = y[:30 * sr]  # First 30 seconds

    # Create a figure for the plots
    plt.figure(figsize=(14, 8))

    # Plot waveform
    plt.subplot(2, 1, 1)
    librosa.display.waveshow(y, sr=sr)
    plt.title('Waveform')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')

    # Compute the spectrogram with smaller n_fft and hop_length for faster computation
    S = librosa.stft(y, n_fft=1024, hop_length=512)
    S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)

    # Plot spectrogram with y-axis in kHz
    plt.subplot(2, 1, 2)
    librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='log', cmap='magma')

    # Convert Hz to kHz by adjusting the colorbar label
    cbar = plt.colorbar(format='%+2.0f dB')
    cbar.set_label('Intensity (dB)')

    plt.title('Spectrogram (dB)')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (kHz)')

    # Display both plots
    plt.tight_layout()
    plt.show()

# User Input: Provide the path to your audio file
audio_file_path = '/content/drive/MyDrive/Cat Song Meter Recordings (1)/Amur Leopard/Dec 15 - Dec 21 7257 inside/SMM07257_20221222_023502.wav'

# Call the function to display the waveform and spectrogram
display_waveform_and_spectrogram(audio_file_path)

KeyboardInterrupt: 